# Simple RNN model with Tensorflow imdb Dataset

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [13]:
# Load the IMDB dataset
max_features = 10000  # Number of unique words to consider(Vocabulary size)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

# Print shapes of the datasets
print(f"Training data shape: {X_train.shape}, Labels shape: {y_train.shape}")
print(f"Test data shape: {X_test.shape}, Labels shape: {y_test.shape}")

Training data shape: (25000,), Labels shape: (25000,)
Test data shape: (25000,), Labels shape: (25000,)


In [14]:
# Inspect sample Reviews
print(f"Sample Review : {X_train[0]}")
print(f"Sample Review Label : {y_train[0]}")

Sample Review : [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
Sample Review Label : 1


In [15]:
# Mapping indexes to words
word_index = imdb.get_word_index()
len(word_index)

88584

In [16]:
reverse_word_index = {v: k for k, v in word_index.items()}

In [17]:
# 🔁 Why i - 3 in next line?
# The original word_index dictionary (from imdb.get_word_index()) maps actual words to integers starting from 1:
# But when Keras processes the dataset, it offsets all word indices by +3 to make room for the reserved tokens above.

# So:
#
# "the" → originally index 1 in word_index
#
# In X_train, "the" will be stored as 1 + 3 = 4
# the first few integer values in the sequences are reserved as follows:
#
# Index	Token
# 0	<PAD> (used for padding)
# 1	<START> (marks the beginning of a review)
# 2	<UNK> (unknown word)
# 3	<UNUSED> (reserved for future use or custom tokens)

decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in X_train[0]])
print(f"Decoded Review: {decoded_review}")

Decoded Review: ? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have do

In [18]:
# Padding sequences to ensure uniform input size
maxlen = 500  # Maximum length of each review
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [23]:
# Train a Simple RNN model
model = Sequential()
model.add(tf.keras.Input(shape=(maxlen,)))
model.add(Embedding(input_dim=max_features, output_dim=128))
model.add(SimpleRNN(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [25]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# create instance for Early Stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [27]:
# Train with Early Stopping
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train, y_train,
    epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 42ms/step - accuracy: 0.5983 - loss: 1.0855 - val_accuracy: 0.7088 - val_loss: 0.5487
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 39s 40ms/step - accuracy: 0.8224 - loss: 0.3997 - val_accuracy: 0.8110 - val_loss: 0.4227
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 40ms/step - accuracy: 0.8820 - loss: 0.2829 - val_accuracy: 0.7806 - val_loss: 0.4760
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 40ms/step - accuracy: 0.9054 - loss: 0.2469 - val_accuracy: 0.8068 - val_loss: 0.4644
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 42s 41ms/step - accuracy: 0.8692 - loss: 39345860608.0000 - val_accuracy: 0.6652 - val_loss: 0.6050
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 41ms/step - accuracy: 0.8426 - loss: 0.3682 - val_accuracy: 0.8086 - val_loss: 0.4519
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 40s 40ms/step - accuracy: 0.9089 - loss: 0.2424 - val_accuracy: 0.8050 - val_loss: 0.4766
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 39ms/step - accuracy: 0.9360 - loss:

In [29]:
model.save('simple_rnn_imdb.keras')